In [ ]:
#| default_exp core

# dialoghelper

In [1]:
#| export
import inspect, json, importlib, linecache
from tempfile import TemporaryDirectory
from fasthtml.common import *
from httpx import get as xget
from fastcore.utils import *
from ghapi.all import *
from fastlite import *

In [89]:
#| export
def get_db():
    if os.environ.get('IN_SOLVEIT', False): dataparent,nm = Path('/app'),'data.db'
    else: dataparent,nm = Path('..'),'dev_data.db'
    db = database(dataparent/'data'/nm)
    all_dcs(db)
    return db

In [90]:
db = get_db()
db.t.dialog.fetchone()

Dialog(id=1, name='test dialog', mode=2)

In [91]:
#| export
def find_var(var: str):
    "Search for var in all frames of the call stack"
    frame = inspect.currentframe()
    while frame:
        dv = frame.f_globals.get(var, frame.f_locals.get(var, None))
        if dv: return dv
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

In [8]:
a = 1
find_var('a')

1

In [92]:
#| export
def find_dialog_id():
    "Get the dialog id by searching the call stack for __dialog_id."
    return find_var('__dialog_id')

In [19]:
__dialog_id = 1

In [20]:
find_dialog_id()

1

In [93]:
#| export
def find_msgs(pattern: str, limit=10):
    "Find messages in a specific dialog that contain the given pattern."
    did = find_dialog_id()
    return db.t.message('did=? AND content LIKE ?', [did, f'%{pattern}%'], limit=limit)

In [22]:
found = find_msgs('to the')
found[0]

Message(id='msg-a2', sid='_vZxms608LW0aPR_nvIlgqQ', content='*Hello* to the **world**!', output='', input_tokens=13, output_tokens=0, msg_type='note', time_run='', is_exported=0, skipped=1, did=1, i_collapsed=0, o_collapsed=0, header_collapsed=0, pinned=0)

In [94]:
#| export
def find_msg_id():
    "Get the message id by searching the call stack for __dialog_id."
    return find_var('__msg_id')

In [28]:
__msg_id = found[0].sid

In [29]:
find_msg_id()

'_vZxms608LW0aPR_nvIlgqQ'

In [95]:
#| export
def read_msg_ids():
    "Get all ids in current dialog."
    did = find_dialog_id()
    return [o.sid for o in db.t.message('did=?', [did], select='sid', order_by='id')]

In [96]:
#| export
def msg_idx():
    "Get index of current message in dialog."
    ids = read_msg_ids()
    return ids,ids.index(find_msg_id())

In [46]:
ids,idx = msg_idx()
idx

2

In [97]:
#| export
def read_msg(n: int=-1,     # Message index (if relative, +ve is downwards)
             relative=True  # Is `n` relative to current message (True) or absolute (False)?
    ):
    "Get the message indexed in the current dialog."
    ids,idx = msg_idx()
    if relative:
        idx = idx+n
        if not 0<=idx<len(ids): return None
    else: idx = n
    return db.t.message.fetchone('sid=?', [ids[idx]])

In [67]:
# Previous message relative to current
read_msg(-1)

Message(id='msg-a1', sid='_I7jB6TkkVt4_sTuQRmhSVw', content='hello world', output='', input_tokens=3, output_tokens=0, msg_type='note', time_run='', is_exported=0, skipped=0, did=1, i_collapsed=0, o_collapsed=0, header_collapsed=0, pinned=0)

In [68]:
# Last message in dialog
read_msg(-1, relative=False)

Message(id='msg-a4', sid='_dZzeZrrPs9ALH5Fjzp-1fw', content='How do I create a new instance?', output='', input_tokens=0, output_tokens=0, msg_type='note', time_run='', is_exported=0, skipped=0, did=1, i_collapsed=0, o_collapsed=0, header_collapsed=0, pinned=0)

In [98]:
#| export
def add_msg(
    content:str, # message that we are updating or adding before/after
    msg_type: str='note', # message type, can be 'code', 'note', or 'prompt'
    output='', # for prompts/code, initial output
    placement='add_after', # can be 'add_after', 'add_before', 'update', 'at_start', 'at_end'
    msg_id=None
):
    "Add/update a message to the queue to show after code execution completes."
    assert msg_type in ('note', 'code', 'prompt'), "msg_type must be 'code', 'note', or 'prompt'."
    kwargs = dict(content=content, msg_type=msg_type, output=output, placement=placement)
    if msg_id: kwargs['msg_id']=msg_id
    run_cmd('add_msg', **kwargs)

In [99]:
#| export
def update_msg(msg):
    "Update an existing message in the dialog."
    add_msg(content=msg.content, msg_type=msg.msg_type, output=msg.output, placement='update', msg_id=msg.id)

In [100]:
#| export
def load_gist(gist_id):
    "Get the first file from a gist"
    api = GhApi()
    if '/' in gist_id: *_,user,gist_id = gist_id.split('/')
    else: user = None
    return api.gists.get(gist_id, user=user)

In [78]:
gistid = 'jph00/e7cfd4ded593e8ef6217e78a0131960c'
gist = load_gist(gistid)
gist.html_url

'https://gist.github.com/jph00/e7cfd4ded593e8ef6217e78a0131960c'

In [101]:
#| export
def gist_file(gist_id):
    "Get the first file from a gist"
    gist = load_gist(gist_id)
    return first(gist.files.values())

In [81]:
gfile = gist_file(gistid)
print(gfile.content)

testfoo='testbar'


In [102]:
#| export
def import_string(code, name):
    with TemporaryDirectory() as tmpdir:
        path = Path(tmpdir) / f"{name}.py"
        path.write_text(code)
        # linecache.cache storage allows inspect.getsource() after tmpdir lifetime ends
        linecache.cache[str(path)] = (len(code), None, code.splitlines(keepends=True), str(path))
        spec = importlib.util.spec_from_file_location(name, path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[name] = module
        spec.loader.exec_module(module)
        return module

In [103]:
#| export
def import_gist(gist_id, mod_name=None, add_global=True):
    "Import gist directly from string without saving to disk"
    fil = gist_file(gist_id)
    mod_name = mod_name or Path(fil['filename']).stem
    module = import_string(fil['content'], mod_name)
    if add_global: inspect.currentframe().f_back.f_globals[mod_name] = module
    return module

In [87]:
import_gist(gistid)
importtest.testfoo

'testbar'

## export -

In [104]:
#|hide
from nbdev import nbdev_export
nbdev_export()